![image.png](https://i.imgur.com/a3uAqnb.png)

## 💡 **Introduction**

In this lab, we explore how to build and train an **Autoencoder for image generation** using **PyTorch**.  
Our goal is to create a generative model that can learn to compress MNIST digit images into a compact latent representation and then reconstruct (or generate) images from these compressed encodings.

#### 🎨 **What is an Autoencoder?**
An autoencoder is a neural network architecture designed to learn efficient data representations in an unsupervised manner. It consists of two main components:

1. **Encoder** 🔽 — Compresses input data into a lower-dimensional latent space (bottleneck)
2. **Decoder** 🔼 — Reconstructs the original data from the compressed representation

#### 🎯 **Our Approach**
1. **Training Phase:**
   - Feed MNIST images (28×28) through the encoder to get compact embeddings
   - Pass embeddings through the decoder to reconstruct images
   - Minimize reconstruction error (MSE Loss) between original and reconstructed images
   
2. **Generation Phase:**
   - Sample random points in the latent space
   - Feed them through the decoder to generate new digit images

#### 🔧 **Architecture Constraints**
In this notebook, we'll use **only Linear (fully connected) layers and BatchNorm** — no CNNs, attention mechanisms, or other advanced techniques. This limitation means:
- Performance may be moderate compared to CNN-based autoencoders
- But it clearly demonstrates the core autoencoder concept!

By the end of this lab, we'll see our model attempting to capture patterns in handwritten digits and generate new samples from random latent codes.

### 🧠 **Importing Libraries and Frameworks**

In this step, we import all the essential Python libraries for our autoencoder implementation:

- **PyTorch** (`torch`) — core deep learning framework for tensor operations and automatic differentiation.  
- **torch.nn** — neural network layers, activations, and loss functions.  
- **NumPy** (`np`) — numerical operations and array handling.  
- **torchvision** — dataset loading and image transformations.  
  - `transforms` — for data augmentation and preprocessing
  - `MNIST` — the handwritten digit dataset
- **torch.utils.data.sampler** (`SubsetRandomSampler`) — for sampling subsets of data.  
- **torchsummary** (`summary`) — provides detailed model architecture summaries.  
- **Matplotlib** (`plt`) — for visualizing training curves and generated images.  
- **mpl_toolkits.axes_grid1** (`ImageGrid`) — for creating organized grids of images.  
- **IPython.display** (`clear_output`) — for cleaning console output.

These imports provide everything we need to build, train, and visualize our autoencoder model.

In [ ]:
from IPython.display import clear_output

In [ ]:
%pip install torch torchvision torchsummary
%pip install numpy
%pip install matplotlib

clear_output()

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

### 📋 **Lab Contents and Autoencoder Overview**

In this notebook, we'll build a **number image generator using autoencoders** trained on the MNIST dataset.

#### 🏗️ **Architecture Overview**

Our model consists of two interconnected parts:

**1. Encoder (Compression Network) 🔽**
- Takes a 28×28 digit image (784 pixels)
- Passes it through multiple linear layers
- Outputs a compact **latent embedding** of specified size (e.g., 128 dimensions)
- **Goal:** Learn meaningful, compressed representations of digits

**2. Decoder (Reconstruction Network) 🔼**
- Takes the latent embedding
- Passes it through multiple linear layers (mirror of encoder)
- Outputs a reconstructed 28×28 image
- **Goal:** Recreate the original image from the compressed representation

#### 🎓 **Training Process**

**Loss Function:** Mean Squared Error (MSE)
```python

### 📊 **Loading MNIST Dataset with Data Augmentation**

In this section, we prepare the MNIST dataset with custom transformations for training.

#### 🔄 **Data Augmentation Pipeline**
```python
transforms.Compose([
    transforms.RandomCrop(28, padding=4),  # Random shifts/translations
    transforms.ToTensor(),                  # Convert to tensor [0, 1]
])
```

**Why Data Augmentation?**
- **RandomCrop(28, padding=4):**
  - Adds 4 pixels of padding around the 28×28 image (making it 36×36)
  - Randomly crops back to 28×28
  - **Effect:** Introduces slight translations and position variations
  - **Benefit:** Model learns position-invariant representations
  
- **ToTensor():**
  - Converts PIL images to PyTorch tensors
  - Normalizes pixel values from [0, 255] to [0.0, 1.0]
  - Changes format to (C, H, W) — PyTorch's expected format

#### 📦 **DataLoader Configuration**
- **Batch Size:** 64 samples per batch
- **Shuffle:** True — randomizes order each epoch for better training
- **Dataset:** 60,000 training images from MNIST

Data augmentation helps the autoencoder learn more robust representations that work across slight variations in digit positioning!

In [ ]:
batch_size = 64

# Define data augmentation pipeline
train_transforms = transforms.Compose([
                        transforms.RandomCrop(28, padding=4),  # Random translation
                        transforms.ToTensor(),                  # Convert to tensor [0, 1]
                    ])

# Load MNIST training data with transformations
train_data = MNIST(root='./datasets', train=True, download=True, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

### 🏗️ **Building the Autoencoder Architecture**

Now we define our `AutoEncoderGenerator` model with encoder and decoder components.

---

#### 🔽 **Encoder Architecture (Compression Network)**

The encoder progressively compresses the 784-dimensional input into a compact latent space:
```
Input (784) → Linear(1024) → LeakyReLU → BatchNorm
           → Linear(512)  → LeakyReLU → BatchNorm
           → Linear(256)  → LeakyReLU
           → Linear(dim_z) → Tanh → Latent Embedding
```

**Layer Breakdown:**
1. **784 → 1024:** Expand first (allows learning complex patterns)
2. **1024 → 512:** Begin compression
3. **512 → 256:** Continue compression
4. **256 → dim_z:** Final bottleneck (e.g., 128 dimensions)

**Activation Functions:**
- **LeakyReLU:** Allows small negative gradients (prevents dying ReLU problem)
- **Tanh (final):** Squashes embeddings to [-1, 1] range
- **BatchNorm:** Normalizes activations for stable training

---

#### 🔼 **Decoder Architecture (Reconstruction Network)**

The decoder mirrors the encoder, expanding from latent space back to image:
```
Latent (dim_z) → Linear(256)  → LeakyReLU
               → Linear(512)  → LeakyReLU
               → Linear(1024) → LeakyReLU
               → Linear(784)  → Sigmoid → Output Image
```

**Layer Breakdown:**
1. **dim_z → 256:** Begin expansion from compact representation
2. **256 → 512:** Continue expansion
3. **512 → 1024:** Further expansion
4. **1024 → 784:** Final reconstruction (28×28 = 784 pixels)

**Activation Functions:**
- **LeakyReLU:** Non-linear transformations during expansion
- **Sigmoid (final):** Squashes output to [0, 1] (valid pixel range)

---

#### 🔄 **Forward Pass Flow**
```python
Input Image (batch, 1, 28, 28)
    ↓ Flatten
(batch, 784)
    ↓ Encoder
Latent Embedding (batch, dim_z)
    ↓ Decoder
Reconstructed (batch, 784)
    ↓ Reshape
Output Image (batch, 1, 28, 28)
```

---

#### 🎯 **Key Design Choices**

1. **Symmetric Architecture:** Decoder mirrors encoder (common in autoencoders)
2. **Bottleneck:** The `dim_z` parameter controls compression level
   - Smaller dim_z → more compression → harder to learn
   - Larger dim_z → less compression → easier but less efficient
3. **No Skip Connections:** Unlike U-Net, we force all information through the bottleneck
4. **Linear Layers Only:** Demonstrates core concept without spatial convolutions

This architecture forces the model to learn efficient, compressed representations of handwritten digits!

In [ ]:
class AutoEncoderGenerator(nn.Module):
  """
  Autoencoder for MNIST digit compression and reconstruction.
  
  Architecture:
  - Encoder: Compresses 784D image to dim_z latent space
  - Decoder: Reconstructs 784D image from latent space
  """
  
  def __init__(self, dim_z):
    super().__init__()
    
    # ============ ENCODER (Compression Network) ============
    self.Encoder = nn.Sequential(
        
        # Layer 1: 784 → 1024 (expand first for pattern learning)
        nn.Linear(28*28, 1024),
        nn.LeakyReLU(),
        nn.BatchNorm1d(1024),
        
        # Layer 2: 1024 → 512 (begin compression)
        nn.Linear(1024, 512),
        nn.LeakyReLU(),
        nn.BatchNorm1d(512),
        
        # Layer 3: 512 → 256 (continue compression)
        nn.Linear(512, 256),
        nn.LeakyReLU(),
        
        # Layer 4: 256 → dim_z (bottleneck/latent space)
        nn.Linear(256, dim_z),
        nn.Tanh(),  # Squash embeddings to [-1, 1]
      )

    # ============ DECODER (Reconstruction Network) ============
    self.Decoder = nn.Sequential(
        
        # Layer 1: dim_z → 256 (begin expansion)
        nn.Linear(dim_z, 256),
        nn.LeakyReLU(),
        
        # Layer 2: 256 → 512 (continue expansion)
        nn.Linear(256, 512),
        nn.LeakyReLU(),
        
        # Layer 3: 512 → 1024 (further expansion)
        nn.Linear(512, 1024),
        nn.LeakyReLU(),
        
        # Layer 4: 1024 → 784 (reconstruct full image)
        nn.Linear(1024, 28*28),
        nn.Sigmoid()  # Squash pixels to [0, 1]
      )

  def forward(self, x):
    """
    Forward pass through encoder and decoder.
    
    Args:
        x: Input images (batch, 1, 28, 28)
    
    Returns:
        Reconstructed images (batch, 1, 28, 28)
    """
    
    original_shape = x.shape
    
    # Flatten image for linear layers: (batch, 1, 28, 28) → (batch, 784)
    x = torch.flatten(x, start_dim=1)
    
    # Encode: (batch, 784) → (batch, dim_z)
    z = self.Encoder(x)
    
    # Decode: (batch, dim_z) → (batch, 784)
    x = self.Decoder(z)
    
    # Reshape back to image format: (batch, 784) → (batch, 1, 28, 28)
    x = x.view(original_shape)
    
    return x

### ⚙️ **Instantiating the Autoencoder Model**

Now we create an instance of our autoencoder with a specific latent dimension.

#### 🎛️ **Latent Dimension (`dim_z`)**
- **Value:** 128 dimensions
- **Meaning:** Each 784-pixel image is compressed into just 128 numbers!
- **Compression Ratio:** 784 / 128 = 6.125× compression

**Effect of Different `dim_z` Values:**
- **Smaller (e.g., 32):** 
  - More aggressive compression
  - Harder to train
  - May lose fine details
  - Better for learning core digit structure

- **Larger (e.g., 256):**
  - Less compression
  - Easier to train
  - Preserves more details
  - Less efficient representation

The 128-dimensional latent space strikes a good balance between compression and reconstruction quality!

In [ ]:
# Set latent dimension (embedding size)
dim_z = 128

# Create autoencoder model
model = AutoEncoderGenerator(dim_z=dim_z)

print(f"✅ Model created with latent dimension: {dim_z}")
print(f"📊 Compression ratio: {28*28 / dim_z:.2f}x")

### 📋 **Model Architecture Summary**

Let's examine our model's architecture in detail using `torchsummary`.

#### 📊 **What the Summary Shows**

The summary provides:
- **Layer-by-layer breakdown** of the architecture
- **Output shapes** after each layer
- **Parameter counts** (trainable weights and biases)
- **Total parameters** and model size

#### 🔍 **Key Metrics to Observe**

1. **Encoder Path:** Watch dimensions shrink from 784 → 128
2. **Decoder Path:** Watch dimensions expand from 128 → 784
3. **Parameter Count:** Total trainable parameters in millions
4. **Memory Usage:** Estimated memory required for forward/backward pass

This summary helps verify:
- Architecture is correctly implemented
- Dimensions flow properly through the network
- Model size is reasonable for our task

In [ ]:
summary(model, (1, 28, 28), device='cpu')

### ⚙️ **Training Configuration and Setup**

Before training, we configure our model, optimizer, loss function, and compute device.

#### 🖥️ **Device Selection (CPU vs GPU)**
```python
device = 'cuda' if torch.cuda.is_available() else 'cpu'
```
- **GPU (CUDA):** Dramatically faster training through parallel computation
- **CPU:** Works everywhere but significantly slower
- Model and all tensors must be on the same device

#### 🎛️ **Hyperparameters**

**Number of Epochs:** 20
- How many times to iterate through the entire dataset
- More epochs → better learning (but risk of overfitting)

**Learning Rate:** 1e-4 (0.0001)
- Controls step size during gradient descent
- Small learning rate for stable training
- Autoencoders typically need careful tuning

#### 🔧 **Optimizer: Adam**
- Adaptive learning rate optimizer
- Combines momentum and RMSprop benefits
- Generally works well with minimal tuning
- Well-suited for autoencoder training

#### 📉 **Loss Function: MSE (Mean Squared Error)**
```python
Loss = MSE(Original, Reconstructed) = mean((Original - Reconstructed)²)
```

**Why MSE for Autoencoders?**
- Measures pixel-wise reconstruction error
- Differentiable and smooth
- Penalizes large errors more than small ones
- Standard choice for image reconstruction tasks

**Alternative Loss Functions:**
- Binary Cross-Entropy: For binary images
- L1 Loss (MAE): For sharper reconstructions
- Perceptual Loss: For better visual quality (requires pre-trained network)

Our setup is now ready for training!

In [ ]:
# Configure device (GPU/CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Move model to device
model.to(device)

# Training hyperparameters
num_epochs = 20
lr = 1e-4

# Optimizer: Adam with learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Loss function: Mean Squared Error for reconstruction
criterion = nn.MSELoss()

print(f"🖥️  Device: {device}")
print(f"📊 Epochs: {num_epochs}")
print(f"📉 Learning Rate: {lr}")
print(f"🎯 Loss Function: MSE")
print("\n✅ Training configuration complete!")

### 🚀 **Training Loop: Learning to Compress and Reconstruct**

Now we train the autoencoder to minimize reconstruction error using batch gradient descent.

#### 🔄 **Training Process (Per Epoch)**

For each epoch, we iterate through all training batches:

**1. Forward Pass:**
```python
X → Encoder → Z (latent) → Decoder → X_hat (reconstructed)
```

**2. Loss Computation:**
```python
Loss = MSE(X, X_hat)  # How different is reconstruction from original?
```

**3. Backward Pass & Optimization:**
```python
optimizer.zero_grad()  # Clear old gradients
loss.backward()        # Compute gradients via backpropagation
optimizer.step()       # Update weights
```

**4. Tracking:**
- Accumulate weighted loss across batches
- Compute average epoch loss

---

#### 🎯 **Training Objective**

The model learns to:
1. **Compress** images into meaningful latent representations
2. **Reconstruct** images as accurately as possible from those representations

**Key Insight:** The bottleneck forces the model to learn the most important features of digits — it can't just memorize!

---

#### 📊 **Expected Behavior**

- **Loss should decrease** steadily over epochs
- **Initial epochs:** Large loss (poor reconstruction)
- **Later epochs:** Low loss (good reconstruction)
- If loss plateaus: Model has converged
- If loss fluctuates wildly: Learning rate may be too high

---

#### ⏱️ **Training Duration**

- **GPU:** ~2-5 minutes for 20 epochs
- **CPU:** ~15-30 minutes for 20 epochs

Let the training begin! 🎓

In [ ]:
# List to track training loss over epochs
train_losses = []

print("🚀 Starting training...\n")

for i in range(num_epochs):
  
  epoch_weighted_loss = 0
  
  for (X, _) in train_loader:  # _ ignores labels (we don't need them!)
    
    # Move batch to device
    X = X.to(device)
    
    # ============ FORWARD PASS ============
    # X_hat because we're trying to reconstruct what's fed to it (X)
    X_hat = model(X)  # X → Encoder → Decoder → X_hat
    
    # ============ COMPUTE LOSS ============
    loss = criterion(X_hat, X)  # MSE between reconstruction and original
    
    # ============ BACKWARD PASS ============
    optimizer.zero_grad()  # Clear gradients from previous batch
    loss.backward()        # Compute gradients
    optimizer.step()       # Update weights
    
    # ============ TRACK LOSS ============
    epoch_weighted_loss += loss.item() * len(X)
  
  # Calculate average loss for epoch
  epoch_loss = epoch_weighted_loss / len(train_loader.dataset)
  train_losses.append(epoch_loss)
  
  # Print progress
  print(f'Epoch {i+1}/{num_epochs}, Loss = {epoch_loss:.6f}')

print("\n✅ Training complete!")

### 📈 **Visualizing Training Progress: Loss Trajectory**

After training, we plot the loss curve to understand the model's learning behavior.

#### 🔍 **Interpretation**

**✅ Ideal Behavior:**
- **Steep decrease** in early epochs (rapid initial learning)
- **Gradual decrease** in later epochs (fine-tuning)
- **Smooth curve** (stable training)
- **Plateau** near the end (convergence)

**⚠️ Potential Issues:**

1. **Loss decreases slowly:**
   - Learning rate too small
   - Model too simple (increase capacity)
   - Need more epochs

2. **Loss remains high:**
   - Architecture limitations (linear layers only!)
   - Bottleneck too small (increase `dim_z`)
   - Complex patterns hard to capture

3. **Loss fluctuates erratically:**
   - Learning rate too large
   - Batch size too small
   - Training instability

---

#### 💡 **Expected Results**

For our linear autoencoder:
- Loss should reach ~0.02-0.05 range
- Won't be perfect (no CNNs for spatial patterns!)
- But good enough to see learned digit structures

Let's see how our model performed! 📊

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, linewidth=2, marker='o', color='#2E86AB')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('Training Curve — Autoencoder Reconstruction Loss', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 🖼️ **Helper Function: Displaying Image Grids**

Before generating images, we define a helper function to display multiple images in an organized grid.

#### 📐 **Function Purpose**

`display_image_grid()` creates a clean, organized visualization of multiple images:
- Arranges images in rows and columns
- Removes axis labels for cleaner appearance
- Adds an overall title
- Uses grayscale colormap for MNIST digits

#### 🎨 **Parameters**
- `images`: List of images to display
- `num_rows`: Number of rows in the grid
- `num_cols`: Number of columns in the grid
- `title_text`: Title for the entire figure

#### 💡 **Why This Helper?**
- Matplotlib's default subplot creation can be verbose
- `ImageGrid` provides cleaner syntax and better spacing
- Reusable for both reconstructions and generations

This makes our result visualization much cleaner and more professional!

In [ ]:
def display_image_grid(images, num_rows, num_cols, title_text):
    """
    Display a grid of images with clean formatting.
    
    Args:
        images: List/array of images to display
        num_rows: Number of rows in the grid
        num_cols: Number of columns in the grid
        title_text: Title for the figure
    """
    
    fig = plt.figure(figsize=(num_cols*3., num_rows*3.))
    grid = ImageGrid(fig, 111, nrows_ncols=(num_rows, num_cols), axes_pad=0.15)
    
    for ax, im in zip(grid, images):
        ax.imshow(im, cmap="gray")
        ax.axis("off")  # Remove axis labels for cleaner look
    
    plt.suptitle(title_text, fontsize=20, fontweight='bold')
    plt.show()

### 🎨 **Generating New Images from Random Latent Codes**

Now comes the exciting part — using our trained decoder to generate completely new digit images!

#### 🎲 **Generation Process**

**1. Sample Random Latent Codes:**
```python
sample_encodings = (torch.rand(N, dim_z) - 0.5) * 2  # Range: [-1, 1]
```
- Generate random values uniformly in [-1, 1]
- Matches the range of our Tanh activation in the encoder
- Each sample is a point in 128-dimensional latent space

**2. Feed Through Decoder:**
```python
generations = model.Decoder(sample_encodings)
```
- **Bypass the encoder entirely!**
- Decoder has learned to map latent space → images
- Each latent code produces a unique image

**3. Reshape for Display:**
```python
generations = generations.reshape(-1, 28, 28, 1)
```
- Convert from (N, 784) to (N, 28, 28, 1)
- Now in image format for visualization

---

#### 🔍 **What to Expect**

**✅ Good Signs:**
- Some images resemble recognizable digits
- Smooth transitions between different digit shapes
- Consistent style across generations

**⚠️ Expected Limitations:**
- Images may be blurry (linear layers only!)
- Some generations may be unclear or ambiguous
- Not all samples will be perfect digits
- Some may be "in-between" digits (e.g., 3-8 hybrid)

**Why?** Our model:
- Uses only linear layers (no spatial awareness)
- Has a compressed latent space
- Samples from random points (not necessarily "valid" digit regions)

---

#### 💡 **Key Insight**

Even with these limitations, if we can identify **any** recognizable digits, it proves our autoencoder has learned meaningful patterns in the latent space!

Let's see what our model generates! 🎨

In [ ]:
# Configuration for generation
rows, cols = 2, 7  # Generate 14 images (2 rows × 7 columns)

# Sample random points in latent space [-1, 1]
sample_encodings = (torch.rand(rows*cols, dim_z).to(device) - 0.5) * 2

print(f"🎲 Sampling {rows*cols} random latent codes...")
print(f"📊 Latent code shape: {sample_encodings.shape}")

# Generate images from random latent codes
model.eval()  # Set to evaluation mode
with torch.no_grad():
    generations = model.Decoder(sample_encodings).cpu()
    generations = generations.reshape(-1, 28, 28, 1)

print("✅ Generation complete!\n")

# Display generated images in a grid
display_image_grid(generations, rows, cols, "Generated Images from Random Latent Codes")

### 🔍 **Interpreting Generated Results**

Let's analyze what our autoencoder has learned and what the generated images tell us.

#### ✅ **What We Can Observe**

**Positive Indicators:**
- Some images show **recognizable digit shapes**
- The model has learned **basic digit structures** (curves, lines, loops)
- Generated images have **consistent style** (grayscale, centered)
- Some samples clearly resemble specific digits (e.g., 0, 1, 6, 8)

**Expected Limitations:**
- Images may be **blurry or unclear**
- Some generations are **ambiguous** (hard to tell which digit)
- Not all samples look like valid digits
- Some may appear as "blends" of multiple digits

---

#### 🧠 **Why These Limitations?**

**1. Linear Layers Only:**
- No convolutional layers for spatial pattern recognition
- No explicit modeling of local pixel relationships
- Each pixel treated independently (not optimal for images)

**2. Limited Training:**
- 20 epochs is relatively short
- More training might improve quality
- But fundamental architectural limitations remain

**3. Random Sampling:**
- We're sampling **uniformly** from [-1, 1]
- True digit representations may cluster in specific regions
- Some random points may be "between" valid digits

---

#### 🎯 **Success Criteria**

Even with limitations, our experiment is successful if:
- ✅ **Any** images are recognizable as digits
- ✅ Model learned **compressed representations**
- ✅ Decoder can **generate** from latent codes
- ✅ Generated images have **digit-like characteristics**

---

#### 💡 **Key Takeaway**

> *"The images aren't perfect, but we can see it's trying to learn. In several images, we can guess which digit it's attempting to generate!"*

This demonstrates that even a simple linear autoencoder can capture meaningful patterns in handwritten digits. The model has successfully learned:
- **Compression:** 784 pixels → 128 dimensions
- **Reconstruction:** 128 dimensions → 784 pixels
- **Generation:** Random codes → digit-like images

With CNN-based architectures, results would be dramatically sharper and more realistic!